# Comparaison des GAN et des VAE pour la génération d'images (sketch)

## Importation des librairies

In [ ]:
import os
import shutil
import tensorflow as tf
import numpy as np

import tensorflow.keras.optimizers as optimizers
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers
from keras.utils import plot_model

In [ ]:
# Dossier source contenant les images
SOURCE_DIR = "./../data/Livrable1/Sketch"
DESTINATION_DIR = "./../data/GAN_DATA"

def organize_images_by_extension(source_dir, destination_dir):
    if not os.path.exists(source_dir):
        print(f"Le dossier {source_dir} n'existe pas.")
        return

    for filename in os.listdir(source_dir):
        file_path = os.path.join(source_dir, filename)

        if os.path.isfile(file_path):
            # Extraire l'extension
            extension = filename.split('.')[-1].lower()

            # Vérifier si c'est bien une image
            if extension in ['jpg', 'jpeg', 'png', 'gif', 'bmp', 'tiff', 'webp']:
                dest_dir = os.path.join(destination_dir, extension)
                os.makedirs(dest_dir, exist_ok=True)

                # Copier le fichier
                shutil.copy(file_path, os.path.join(dest_dir, filename))
                print(f"Copié : {filename} -> {dest_dir}")

# organize_images_by_extension(SOURCE_DIR, DESTINATION_DIR)


In [ ]:
# Charger les images
def load_dataset(data_dir, IMG_SHAPE, batch_size, VALIDATION_SPLIT, SEED):
    # Charger le dataset à partir des répertoires
    train_set = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=VALIDATION_SPLIT,
        subset='training',  
        seed=SEED, 
        labels=None,
        image_size=IMG_SHAPE,
        batch_size=batch_size,
        shuffle=True,
    )

    test_set = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=VALIDATION_SPLIT,
        subset='validation',  
        seed=SEED,
        labels=None,
        image_size=IMG_SHAPE,
        batch_size=batch_size,
        shuffle=True,
    )
    return train_set, test_set

# Fonctions création GAN

In [ ]:
def build_generator(LATENT_DIM, USE_BIAIS, IMG_HEIGHT, IMG_WIDTH):
    model = tf.keras.Sequential([
        layers.Dense(IMG_HEIGHT // 8 * IMG_WIDTH // 8 * 256, 
                     use_bias=USE_BIAIS, 
                     input_shape=(LATENT_DIM,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Reshape((IMG_HEIGHT // 8, IMG_WIDTH // 8, 256)),

        layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding="same", use_bias=USE_BIAIS),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding="same", use_bias=USE_BIAIS),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding="same", activation="tanh"),
    ])
    return model

In [ ]:
def build_discriminator(IMG_SHAPE):
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding="same", input_shape=IMG_SHAPE),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Conv2D(128, (5, 5), strides=(2, 2), padding="same"),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1, activation="sigmoid"),
    ])
    return model

In [ ]:
# Optimizers
generator_optimizer = optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
discriminator_optimizer = optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

cross_entropy = tf.keras.losses.BinaryCrossentropy()

# Fonction de perte
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

# Boucle d'entraînement
@tf.function
def train_step(images, generator, discriminator, BATCH_SIZE, LATENT_DIM):
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))

    return gen_loss, disc_loss

def train(dataset, generator, discriminator, BATCH_SIZE, LATENT_DIM, epochs=50):
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        for image_batch in dataset:
            train_step(image_batch, generator, discriminator, BATCH_SIZE, LATENT_DIM)
        


In [ ]:
def generate_and_show_images(generator, LATENT_DIM):
    noise = tf.random.normal([16, LATENT_DIM])
    generated_images = generator(noise, training=False)

    fig, axes = plt.subplots(4, 4, figsize=(4, 4))
    for i, ax in enumerate(axes.flat):
        ax.imshow((generated_images[i] + 1) / 2)  # Dé-normalisation
        ax.axis("off")
    plt.show()

# Fonctions création VAE

In [ ]:
# Coding the specific sampling layer as a Keras Layer object
class Sampling(layers.Layer):
    """
    Create a custom sampling layer for the VAE.
    This layer takes the mean and log variance of the latent space
    and defines z as a random variable sampled from the normal distribution. 
    """
    def call(self, inputs):
        z_mean, z_logvar = inputs

        nbatch = K.shape(z_mean)[0]
        ndim = K.shape(z_mean)[1]

        std = K.exp(z_logvar / 2)  # Correction: diviser par 2 pour obtenir l'écart-type
        eps = K.random_normal(shape=(nbatch, ndim), mean=0., stddev=1.0)  # Correction: stddev=1.0

        z = z_mean + eps * std

        return z

In [ ]:
def create_encoder_and_decoder(LATENT_DIM, IMG_SHAPE=(256, 256,)):
    
    """
    Create the encoder and decoder models for the VAE.
    """
    # ------------------ Encoder -----------------
    encoder_inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = layers.Conv2D(32, (3, 3), strides=2, padding="same")(encoder_inputs)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(64, (3, 3), strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
        
    x = layers.Conv2D(128, (3, 3), strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)


    x = layers.Conv2D(256, (3, 3), strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)

    x = layers.Dropout(0.3)(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(300, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    
    z_mean = layers.Dense(LATENT_DIM, kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    z_logvar = layers.Dense(LATENT_DIM, kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    
    encoder = tf.keras.Model(encoder_inputs, [z_mean, z_logvar], name="encoder")
    
    
    # ------------------ Decoder -------------------
    IMG_HEIGHT, IMG_WIDTH, _ = IMG_SHAPE
    decoder_inputs = tf.keras.Input(shape=(LATENT_DIM,))
    
    # Étendre le vecteur latent à une taille appropriée
    decoder_hidden = layers.Dense((IMG_HEIGHT // 8) * (IMG_WIDTH // 8) * 256, activation="relu")(decoder_inputs)
    reshaped_hidden = layers.Reshape((IMG_HEIGHT // 8, IMG_WIDTH // 8, 256))(decoder_hidden)

    # Reconstruction progressive de l'image
    x = layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding="same")(reshaped_hidden)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.25)(x)
    
    x = layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding="same", activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.25)(x)
    
    decoder_outputs = layers.Conv2DTranspose(3, (3, 3), strides=(2, 2), padding="same", activation="sigmoid")(x)
    decoder_outputs = layers.Reshape((IMG_HEIGHT, IMG_WIDTH, 3))(decoder_outputs)

    decoder = tf.keras.Model(decoder_inputs, decoder_outputs, name="decoder")
    
    return encoder, decoder

In [ ]:
class VAE_Autoencoder(tf.keras.Model):
    def __init__(self, input_dim, encoder, decoder, beta=0.001):
        super(VAE_Autoencoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.input_dim = input_dim
        self.beta = beta  # Weight for the KL divergence loss

    def call(self, inputs):
        # Encode inputs to latent space
        z_mean, z_logvar = self.encoder(inputs)
        z = Sampling()([z_mean, z_logvar])  # Sample latent vector
        reconstructed = self.decoder(z)  # Decode latent vector

        # Reconstruction loss
        reconstruction_loss = tf.keras.losses.binary_crossentropy(inputs, reconstructed)
        reconstruction_loss = tf.reduce_sum(reconstruction_loss, axis=(1, 2))  # Sum over spatial dimensions
        reconstruction_loss = tf.reduce_mean(reconstruction_loss)  # Average over the batch

        # Clip z_mean and z_logvar for numerical stability
        z_mean = tf.clip_by_value(z_mean, -5.0, 5.0)  # Clipping plus strict
        z_logvar = tf.clip_by_value(z_logvar, -5.0, 5.0)  # Clipping plus strict

        # KL divergence loss
        epsilon = 1e-8  # Small constant for numerical stability
        kl_loss = -0.5 * tf.reduce_sum(1 + z_logvar - tf.square(z_mean) - tf.exp(z_logvar + epsilon), axis=-1)
        kl_loss = tf.reduce_mean(kl_loss)  # Average over the batch

        # Total VAE loss
        vae_loss = reconstruction_loss + self.beta * kl_loss
        self.add_loss(vae_loss)  # Add the loss to the model

        return reconstructed

# Comparaison GAN vs VAE

In [ ]:
BATCH_SIZE = 16
DATASET_PATH = "./../data/GAN_DATA/jpg"
SEED = 42
LATENT_DIM = 20  # Taille du bruit aléatoire
VALIDATION_SPLIT = 0.2  # Pourcentage de données pour l'entraînement
USE_BIAIS = True
IMG_HEIGHT = 576 # real size : 583 - rounded to be divisible by 8
IMG_WIDTH = 408 # real size : 411 - rounded to be divisible by 8
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

GAN_EPOCHS = 100
VAE_EPOCHS = 100

## Création du dataset

In [ ]:
# Création du dataset TensorFlow

train_dataset, test_dataset = load_dataset(
    data_dir=DATASET_PATH,
    IMG_SHAPE=IMG_SHAPE[:2],  # Pass only height and width
    batch_size=BATCH_SIZE,
    VALIDATION_SPLIT=VALIDATION_SPLIT,
    SEED=SEED,
)

In [ ]:
train_dataset = train_dataset.map(lambda x: x / 255.0)  # Normalize to [0, 1]
test_dataset = test_dataset.map(lambda x: x / 255.0)  # Normalize to [0, 1]

for dataset in [train_dataset, test_dataset]:
    for batch in dataset.take(1):
        min_val = tf.reduce_min(batch).numpy()
        max_val = tf.reduce_max(batch).numpy()
        
        assert min_val >= 0 and max_val <= 1, "Min and Max values are not in the range [0, 1]"
        print("Test Dataset - Min:", min_val, "Max:", max_val)    

In [ ]:
# prefetch
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

## Création du GAN

In [ ]:
generator = build_generator(LATENT_DIM, USE_BIAIS, IMG_HEIGHT, IMG_WIDTH)

In [ ]:
discriminator = build_discriminator(IMG_SHAPE)

In [ ]:
train(train_dataset, 
      epochs=GAN_EPOCHS, 
      generator=generator, 
      discriminator=discriminator, 
      BATCH_SIZE=BATCH_SIZE, 
      LATENT_DIM=LATENT_DIM)

In [ ]:
# generate_and_show_images(generator=generator, LATENT_DIM=LATENT_DIM)

## Création du VAE

In [ ]:
"""
Create encoder and decoder models for the VAE
"""
encoder, decoder = create_encoder_and_decoder(LATENT_DIM, IMG_SHAPE)
plot_model(encoder, to_file='./figures/encoder.png', show_shapes=True)
plot_model(decoder, to_file='./figures/decoder.png', show_shapes=True)

In [ ]:
encoder.summary()
decoder.summary()

In [ ]:
vae = VAE_Autoencoder(input_dim=IMG_SHAPE, 
                      encoder=encoder, 
                      decoder=decoder, 
                      beta=0.001)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, clipnorm=1.0)  # Ajout de gradient clipping
vae.compile(optimizer=optimizer, loss=None)
vae.summary()

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        verbose=1,
        mode='min',
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='./models/vae_model.keras',
        save_best_only=True,
        monitor='val_loss',
        mode='min'
    ),]

history_vae = vae.fit(
    train_dataset,
    epochs=VAE_EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True,
    validation_data=test_dataset,
    verbose=1,
    callbacks=callbacks,
)

In [ ]:
img = vae.predict(
    test_dataset.take(1),
    batch_size=BATCH_SIZE,
)
# Plotting images
plt.figure(figsize=(8, 8))
plt.imshow(img[1])
plt.axis("off")
    
plt.show()

In [ ]:
test_images = test_dataset.take(1)
for batch in test_images:
    test_images = batch.numpy()
    break
vae_images = vae.predict(test_images)

In [ ]:
# Crée une figure avec trois sous-grilles de 4x4
fig, axes = plt.subplots(2, 9, figsize=(12, 6))

# Affiche les images du test dataset dans la première sous-grille
for i, ax in enumerate(axes[0]):
    ax.imshow(test_images[i], cmap='gray')
    ax.axis('off')
    if i == 0:
        ax.set_title('Test Dataset')

# Affiche les images générées par le VAE dans la deuxième sous-grille
for i, ax in enumerate(axes[1]):
    ax.imshow(vae_images[i], cmap='gray')
    ax.axis('off')
    if i == 0:
        ax.set_title('VAE Images')

# Ajuste l'espacement entre les sous-grilles
plt.tight_layout()
plt.show()

In [ ]:
generate_and_show_images(generator=generator, LATENT_DIM=LATENT_DIM)